In [5]:
#following tutorials to test and work with CNNs and Pytorch functions for practice
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [6]:

transform = transforms.Compose([
    transforms.ToTensor(), #transforming rgb into -1 to 1 to train faster
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) 

In [7]:
train_data = torchvision.datasets.CIFAR10(root = './data', train = True, transform = transform, download = True)
test_data = torchvision.datasets.CIFAR10(root = './data', train = False, transform = transform, download = True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True, num_workers = 2) #each step 32 images and 32 labels
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = True, num_workers = 2)


In [8]:
image, label = train_data[0]
image
label

6

In [9]:
image.size() #3 channels, 32 by 32 pixels

torch.Size([3, 32, 32])

In [10]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [11]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        # convulational layers
        self.conv1 = nn.Conv2d(3, 12, 5) #5 times 5 kernal moving across features producing 12 feature maps
        self.pool = nn.MaxPool2d(2, 2) # 12, 14, 12
        self.conv2 = nn.Conv2d(12, 24, 5) #14 - 5 /1 = 9 +1 = 10

        #fully connected dense layers
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #fed through a relu activation funct
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) #flatten into a single vector per image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

In [ ]:
for epoch in range(40):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0 #cumulative loss for this epoch
    
    for i, data in enumerate(train_loader): #go over mini bacthes
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs) #forward pass
        loss = loss_function(outputs, labels)
        loss.backward() #backward pass
        optimizer.step() #udpdate the weights

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.2106
Training epoch 1...
Loss: 1.7424
Training epoch 2...


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

In [ ]:
correct = 0 
total = 0

net.eval()
with torch.no_grad(): #turn off autograd for now
    for data in test_loader: #loop over the mini-batches
        images, labels = data
        outputs = net(images) #forward pass
        _, predicted = torch.max(outputs, 1) #take the index of the larget logit across the dimention 
        total += labels.size(0) #add batch size to total
        correct += (predicted == labels).sum().item() #count batches and element wise comparision

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

In [ ]:
img_idx = 32                    
image, true_label = test_data[img_idx]

input_tensor = image.unsqueeze(0)

net.eval()
with torch.no_grad():
    logits = net(input_tensor)               # forward pass
    pred_idx = logits.argmax(dim=1).item()   # get class with highest score

pred_label = class_names[pred_idx]
true_label_name = class_names[true_label]

print(f"Model prediction : {pred_label}")
print(f"Ground truth     : {true_label_name}")

img_to_show = image * 0.5 + 0.5              # put pixels back into [0,1]
plt.imshow(img_to_show.permute(1, 2, 0))     # C×H×W ➜ H×W×C for matplotlib
plt.title(f"Predicted: {pred_label} | True: {true_label_name}")
plt.axis("off")
plt.show()